In [191]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [178]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline

In [179]:
%run query_features.py
%run scripts/helper.py
%run scripts/model_train_plus_test.py

In [180]:
crowd_train = load_file('./data/train.csv/train.csv', None)
crowd_test = load_file('./data/test.csv/test.csv', None)

In [181]:
# crowd_train = crowd_train[crowd_train.relevance_variance < 0.5]

In [158]:
target = crowd_train.median_relevance.values

In [182]:
# train_index, test_index = ssSplit(target, train_size=8000, random_state=44)
# train_index, test_index = ssSplit(target, train_size=500, random_state=44)

In [183]:
Xt = crowd_train.iloc[train_index]
Xv = crowd_train.iloc[test_index]

# Xt = crowd_train
# Xv = crowd_test

In [184]:
yt = target[train_index]
yv = target[test_index]

# yt = target

In [185]:
correct_map = build_query_correction_map(Xt, crowd_test)

In [186]:
def spell_correct_query(x):
    if x not in correct_map:
        return x
    else:
        return correct_map[x]
    
Xt['query'] = Xt['query'].map(spell_correct_query)

In [187]:
Xv['query'] = Xv['query'].map(spell_correct_query)

In [188]:
Xt_tweaked = tweak_text(Xt)
Xv_tweaked = tweak_text(Xv)

In [189]:
Xfitted, tfv = TFIDF(Xt_tweaked, None)

In [192]:
svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)

scl = StandardScaler(copy=True, with_mean=True, with_std=True)

clf = SVC(C=10.0, kernel='rbf', degree=3, 
        gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
        tol=0.001, cache_size=200, class_weight=None, 
        verbose=False, max_iter=-1, random_state=None)

keywords = keyword_counter(Xt)

In [193]:
features = stack([keywords, Xfitted])

In [194]:
pipeline = Pipeline([('svd', svd), ('scl', scl), ('clf', clf)])

In [195]:
pipeline.fit(features, yt)

Pipeline(steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
       random_state=None, tol=0.0)), ('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [196]:
keywords_test = keyword_counter(Xv)

In [197]:
Xtest = tfv.transform(Xv_tweaked)

In [198]:
features_test = stack([keywords_test, Xtest])

In [199]:
preds_new_model = pipeline.predict(features_test)

In [148]:
# make_submission(crowd_test.id.values.astype(int), preds_new_model, 'spell_correct_rel.csv')

In [200]:
print 'Kappa score on validation set ', (quadratic_weighted_kappa(yv, preds_new_model))

### Linear model

In [174]:
linear_model, select = build_linear_model(features, yt)

In [175]:
features_test_selected = select.transform(features_test)
linear_preds = linear_model.predict(features_test_selected)

In [152]:
# print 'Kappa score on validation set ', (quadratic_weighted_kappa(yv, linear_preds))

Kappa score on validation set  0.57412956621


In [176]:
ensemble_lin_svm = (preds_new_model + linear_preds) / 2

In [155]:
# print 'Kappa score on validation set ', (quadratic_weighted_kappa(yv, ensemble_lin_svm))

Kappa score on validation set  0.659703361754


In [177]:
make_submission(crowd_test.id.values.astype(int), ensemble_lin_svm, 'ensemble_lin_svm_title.csv')

## Best score

In [48]:
best_score_df = pd.read_csv('./submissions/spell_correct_final_only_title.csv')

In [49]:
best_score = best_score_df.prediction

In [50]:
ensemble = (preds_new_model + best_score) / 2
ensemble_int = [int(score) for score in ensemble]

In [51]:
make_submission(crowd_test.id.values.astype(int), ensemble_int, 'spell_correct_title_relevance.csv')